In [107]:
import pandas as pd
import glob
import os

# Data converter

- Data comes from XL in three formats, and put into the folder excel_data.

- Data can also come from csv's from csv_folder

- All data is converted to the right formatting as a csv inside final_etf_data

In [108]:
etf_files_path = 'final_etf_csv'

## Converts all XL data to correctly format csv files

In [109]:
excel_files_path = 'excel_data/format_'
num_formats = 3

In [110]:
format_1 = [
    'GlobalPE.xlsx',
    'STOXX600.xlsx'
]

format_2 = [
    'FTSE EMU Government Bond (EGBI) TRI.xlsx',
    'FTSECORP.xlsx',
    'FTSECORPALT.xlsx',
    'FTSEGOV.xlsx',
    'FTWBIGUSDT.xlsx'
    
]

format_3 = [
    'FTSEREEL.xlsx',
    'MSCI World.xlsx',
    'PE.xlsx',
    'FTSE EPRA Nareit Developed Europe.xlsx',
    'LPX50PE.xlsx',
    'MSCI ACWI.xlsx',
    'MSCI EUROPE.xlsx',
    'MSCI EMU.xlsx',
]

In [111]:
# Deletes every csv in final_etf_csv
files = glob.glob(os.path.join(etf_files_path, '*'))

for file in files:
    try:
        os.remove(file)
        print(f"Deleted: {file}")
    except Exception as e:
        print(f"Error deleting {file}: {e}")

Deleted: final_etf_csv/FTSEREEL.csv
Deleted: final_etf_csv/LPX50PE.csv
Deleted: final_etf_csv/GlobalPE.csv
Deleted: final_etf_csv/MSCI World.csv
Deleted: final_etf_csv/FTSECORP.csv
Deleted: final_etf_csv/PE.csv
Deleted: final_etf_csv/MSCI ACWI.csv
Deleted: final_etf_csv/FTWBIGUSDT.csv
Deleted: final_etf_csv/FTSE EPRA Nareit Developed Europe.csv
Deleted: final_etf_csv/MSCI EMU.csv
Deleted: final_etf_csv/STOXX600.csv
Deleted: final_etf_csv/FTSECORPALT.csv
Deleted: final_etf_csv/FTSE EMU Government Bond (EGBI) TRI.csv


In [112]:
for i in range(1, 1 + num_formats):
    formated_files_path = excel_files_path + str(i)
    for filename in os.listdir(formated_files_path):
        if filename.endswith(".xlsx"):
            file_path = os.path.join(formated_files_path, filename)

            if filename in format_1:
                df = pd.read_excel(file_path, header=None)
                keyword_row_index = df[df.iloc[:, 0] == 'Exchange Date'].index[0]

                df = pd.read_excel(file_path, skiprows=keyword_row_index)
                df = df[['Exchange Date', 'Close']]
                df.dropna(subset=['Exchange Date', 'Close'], inplace=True)
                df.rename(columns={'Exchange Date' : 'Date', 'Close': 'Price'}, inplace=True)

            elif filename in format_2:
                df = pd.read_excel(file_path, skiprows=10)  
                df = df[['Date', 'Trade Price']]
                df.dropna(subset=['Date', 'Trade Price'], inplace=True)
                df.rename(columns={'Trade Price': 'Price'}, inplace=True)

            elif filename in format_3:
                df = pd.read_excel(file_path, skiprows=17)
                df = df[['Exchange Date', 'Close']]
                df.dropna(subset=['Exchange Date', 'Close'], inplace=True)
                df.rename(columns={'Exchange Date' : 'Date', 'Close': 'Price'}, inplace=True)
            
            else:
                continue
            
            csv_file_path = os.path.join(etf_files_path, f"{os.path.splitext(filename)[0]}.csv")
            
            # Save to CSV
            df.to_csv(csv_file_path, index=False)
            
            print(f"Converted {filename} to CSV and saved as {csv_file_path}")

Converted STOXX600.xlsx to CSV and saved as final_etf_csv/STOXX600.csv
Converted GlobalPE.xlsx to CSV and saved as final_etf_csv/GlobalPE.csv
Converted FTSECORPALT.xlsx to CSV and saved as final_etf_csv/FTSECORPALT.csv
Converted FTWBIGUSDT.xlsx to CSV and saved as final_etf_csv/FTWBIGUSDT.csv
Converted FTSE EMU Government Bond (EGBI) TRI.xlsx to CSV and saved as final_etf_csv/FTSE EMU Government Bond (EGBI) TRI.csv
Converted FTSECORP.xlsx to CSV and saved as final_etf_csv/FTSECORP.csv
Converted PE.xlsx to CSV and saved as final_etf_csv/PE.csv
Converted FTSEREEL.xlsx to CSV and saved as final_etf_csv/FTSEREEL.csv
Converted MSCI ACWI.xlsx to CSV and saved as final_etf_csv/MSCI ACWI.csv
Converted LPX50PE.xlsx to CSV and saved as final_etf_csv/LPX50PE.csv
Converted MSCI World.xlsx to CSV and saved as final_etf_csv/MSCI World.csv
Converted FTSE EPRA Nareit Developed Europe.xlsx to CSV and saved as final_etf_csv/FTSE EPRA Nareit Developed Europe.csv
Converted MSCI EMU.xlsx to CSV and saved a

## Converts imported CSVs into correct format in the right folder

In [113]:
csv_file_path = 'csv_data'

In [ ]:
csv_files = [
    #'ICE BofA EBL0 (Euro Large Cap Corporate).csv',
    'ICE BofA ER00 (Euro Corporate).csv',
    'ICE BofA G0BC (Corporate World).csv',
    'ICE BofA HE00 (Euro HY).csv',
    'ICE BofA HW00 (Global HY).csv',
    #'MSCI World Index.csv',
]

In [115]:
for file in os.listdir(csv_file_path):
    if file.endswith('.csv'):
        file_path = os.path.join(csv_file_path, file)
        
        with open(file_path, 'r', encoding='utf-8') as f:
            first_line = f.readline().strip()
        
        if first_line.startswith("sep="):
            sep = first_line.split('=')[1]
            df = pd.read_csv(file_path, sep=sep, skiprows=1)
        else:
            df = pd.read_csv(file_path, sep=';')

        # Process Format 1: ["Date"; "Line(Q.MERER00)"]
        if df.columns[0] == 'Date' and len(df.columns) == 2:
            df.columns = ['Date', 'Price']
            df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y', errors='coerce')
            
            df['Price'] = df['Price'].apply(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)
            df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

        # Process Format 2: ["Date;Open;High;Low;Last;Volume;Status"]
        elif 'Date' in df.columns and 'Last' in df.columns:
            df = df[['Date', 'Open']]
            df.rename(columns={'Open': 'Price'}, inplace=True)
            df.columns = ['Date', 'Price']
            df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y', errors='coerce')
            
            df['Price'] = df['Price'].apply(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)
            df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

        else:
            print(f"Skipping unrecognized format in file: {file}")
            continue

        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

        output_file_path = os.path.join(etf_files_path, file)
        df.to_csv(output_file_path, index=False, sep=',', header=['Date', 'Price'])
        print(f"Processed and saved: {output_file_path}")

EmptyDataError: No columns to parse from file